# Effect of Infill Patterns on Sintering

#### Introduction
Metal 3D printing, also known as additive manufacturing, revolutionizes traditional manufacturing by enabling the direct fabrication of metal parts from digital designs. Unlike conventional methods that involve subtractive processes (removing material) or casting, metal 3D printing builds up parts layer by layer.

#### Binder-Based Metal Printing
One prominent method in metal 3D printing involves the use of a binder-based process. Here, metal powder is combined with a binder material to form a feedstock or paste. This mixture is then precisely deposited layer by layer according to the design specifications.

#### The 'Green' State
After printing, the object is in a 'green' state. At this stage, it is formed from metal particles held together by the binder. The part is relatively fragile and lacks the full mechanical properties of the intended metal due to the binder's presence.

#### Sintering Process
To transform the green part into a robust metal object, it undergoes a crucial post-processing step known as sintering. Sintering involves heating the green part in a controlled atmosphere furnace. During this process, the binder material evaporates or burns off, and the metal particles fuse together. This fusion strengthens the part, improving its mechanical properties and durability.

<center>
    <img src="img/01.gif" alt="Alt Text" width="400"/>
</center>


#### Problem
During sintering, the binder evaporates or burns off, leaving voids where it was originally present. The density of these voids and the overall distribution of metal particles directly impact the final mechanical properties and dimensional accuracy of the sintered part. Different infill patterns and densities in the printed object can lead to varying results in terms of porosity, strength, and other critical characteristics of the printed metal objects post-sintering.

#### Objective
This study module aims to extract actionable data from different infill patterns and densities in metal 3D printing with binder paste. The focus is on gathering empirical insights that can serve as inputs for predictive models of shrinkage during the sintering process. By systematically testing various infill configurations, we seek to identify correlations between pattern types, densities, and resulting shrinkage rates. This approach will provide valuable guidance for optimizing the design and fabrication of metal parts, ensuring better dimensional accuracy and performance in diverse industrial applications.

### Interactive Selection of Infill Patterns and Percentages
Different infill patterns in metal 3D printing vary not only in their geometric shapes but also in their orientations during printing. For instance, traditional grid patterns maintain the same orientation throughout layers, providing consistent structural support. In contrast, patterns like Triangles rotate by 90 degrees with each layer, alternating orientation to enhance part strength and reduce anisotropy. More complex patterns such as Gyroid exhibit intricate rotations and orientations across layers, optimizing material distribution for improved mechanical properties and reduced material usage.

In the cell below, you have the capability to select any infill pattern from a dropdown menu and adjust the infill percentage according to your needs. This interactive setup allows you to observe and analyze how different infill patterns and their corresponding densities affect the printing process across various layers. Explore various combinations to see if and how changes in infill pattern and percentage influence the structural and visual aspects of the print through different layers.

#### Press ▶️ to select pattern shape and infill density

In [ ]:
import os
from functions import*
from ipywidgets import interact_manual, Dropdown
data = None
# Directory path where patterns are stored
patterns_folder = './patterns'

# List of all possible pattern names and infill percentages
pattern_names = [
    'Cross', 'Grid', 'Gyroid', 'Octet', 'QuarterCube',
    'Triangle', 'TriHex', 'ZigZag'
]

infill_percentages = ['10%', '25%', '40%', '50%', '60%', '75%', '90%', '100%']

# Function to load data based on selected pattern name and infill percentage
def load_data(pattern_name, infill_percentage):
    global data
    # Standardize the pattern name to match the file naming convention
    pattern_name_formatted = pattern_name.lower().replace(' ', '').replace('quartercubic', 'quartercube')
    
    # Remove the '%' character from the infill percentage
    infill_percentage_value = int(infill_percentage.replace('%', '')) / 100.0
    infill_percentage_formatted = f"{infill_percentage_value:.2f}"
    
    # Construct the filename using the adjusted pattern name and infill percentage
    file_name = f"{pattern_name_formatted}_{infill_percentage_formatted}.gcode"
    file_path = os.path.join(patterns_folder, file_name)
    data = parse_gcode(file_path, slicer_type = 'cura') # can also be 'prusa'
    print('Pattern Selected ✓')


# Dropdown widgets for pattern name and infill percentage
pattern_dropdown = Dropdown(options=pattern_names, description='Pattern Name:')
infill_dropdown = Dropdown(options=infill_percentages, description='Infill Percentage:')

# Interact_manual to tie widgets with data loading function
interact_manual(load_data, pattern_name=pattern_dropdown, infill_percentage=infill_dropdown, button_description="Load File");

#### Press ▶️ to observe different orientations associated with patterns
(Adjust the layer height using the provided slider to observe any changes in the infill pattern's impact on the print across different layer heights)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from ipywidgets import interact, Dropdown

# Filtered unique Z values where E > 0
z_values = sorted(data.loc[data['E'] > 0, 'Z'].unique())

# Function to plot based on selected Z value
def plot_layer(z_value):
    fig = plt.figure(figsize=(5, 5))
    ax = fig.add_subplot(111)
    ax.axis('off')

    for i in range(len(data) - 1):
        if data['E'].iloc[i] > 0 and data['Z'].iloc[i] == z_value:
            x_values = [data['X'].iloc[i], data['X'].iloc[i + 1]]
            y_values = [data['Y'].iloc[i], data['Y'].iloc[i + 1]]
            ax.plot(x_values, y_values, color='black', linewidth=5.6693)

    ax.set_title(f'Layer Z = {z_value}')
    ax.set_aspect(aspect='equal')
    plt.show()

# Dropdown widget for selecting Z values
dropdown = Dropdown(options=z_values, description='Select Z value:')

# Interact to tie widget with plotting function
interact(plot_layer, z_value=dropdown);

Two key methods are employed to extract crucial data during analysis. 

- First, detecting overlapping regions between consecutive layers helps assess the continuity and structural robustness provided by different infill patterns. This analysis is essential for understanding how well the layers integrate and support each other, crucial for mitigating anisotropy in shrinkage and ensuring uniform mechanical properties across the printed part.

- Second, gravity impact analysis identifies regions in each layer that lack direct support from the layer beneath, highlighting potential weak points susceptible to deformation or failure. By pinpointing these unsupported areas, engineers can refine infill patterns and printing parameters to enhance overall print quality and durability, addressing structural weaknesses before final sintering and assembly.

<center>
    <img src="img/01.jpg" alt="Alt Text" width="700"/>
</center>


#### Press ▶️ to run pattern feature generation

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import pandas as pd
from io import BytesIO
from ipywidgets import SelectMultiple, RadioButtons, VBox, Button, Output
from IPython.display import display, clear_output, Image

def plot_layer_image(z_value):
    """Generates and returns an image for a given layer."""
    fig, ax = plt.subplots(figsize=(5, 5))
    ax.axis('off')
    layer_data = data[(data['Z'] == z_value) & (data['E'] > 0)]
    for i in range(len(layer_data) - 1):
        x_values = [layer_data['X'].iloc[i], layer_data['X'].iloc[i + 1]]
        y_values = [layer_data['Y'].iloc[i], layer_data['Y'].iloc[i + 1]]
        ax.plot(x_values, y_values, color='black', linewidth=5.6693)
    ax.set_aspect('equal')
    fig.tight_layout(pad=-1.5)
    
    buf = BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    img_array = np.frombuffer(buf.getvalue(), dtype=np.uint8)
    buf.close()
    plt.close(fig)
    
    image = cv2.imdecode(img_array, cv2.IMREAD_GRAYSCALE)
    _, binary_image = cv2.threshold(image, 240, 255, cv2.THRESH_BINARY_INV)
    
    return binary_image

def add_layers(layers):
    """Adds images of multiple layers."""
    if not layers:
        return None
    base_image = plot_layer_image(layers[0])
    for z_value in layers[1:]:
        image = plot_layer_image(z_value)
        base_image = cv2.bitwise_or(base_image, image)
    return base_image

def subtract_layers(layers):
    """Subtracts images of multiple layers."""
    if not layers:
        return None
    base_image = plot_layer_image(layers[0])
    for z_value in layers[1:]:
        image = plot_layer_image(z_value)
        base_image = cv2.bitwise_and(base_image, cv2.bitwise_not(image))
    return base_image

# Widgets
layers_dropdown = SelectMultiple(options=sorted(data['Z'].unique()), description='Select Layers:')
operation_select = RadioButtons(options=['add', 'subtract'], description='Operation:')
run_button = Button(description="Process Layers")
display_output = Output()  # Changed variable name from output to display_output

vbox = VBox([layers_dropdown, operation_select, run_button, display_output])
display(vbox)

def on_button_clicked(b):
    with display_output:
        clear_output(wait=True)  # Clears the previous output to keep the display clean
        layers = list(layers_dropdown.value)
        if not layers:
            print("No layers selected.")
            return
        
        if operation_select.value == 'add':
            result_image = add_layers(layers)
        elif operation_select.value == 'subtract':
            result_image = subtract_layers(layers)
        
        if result_image is not None:
            result_image = cv2.bitwise_not(result_image)  # Invert the colors
            _, buffer = cv2.imencode('.png', result_image)
            display(Image(data=buffer))
    
run_button.on_click(on_button_clicked);


### Analysis of Extracted Data

This data was extracted from all infill patterns, providing critical insights into the structural aspects of 3D printed objects. 
The interactive chart below illustrates how different infill patterns and densities affect these extracted features, offering a visual representation of the impact on structural integrity and support across layers.


#### Press ▶️ to run data analysis and select multiple pattern data

In [ ]:
import pandas as pd
import os
import re
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

# Sample directory containing your CSV files
data_folder = 'values'

# List of all CSV filenames
csv_files_list = [file for file in os.listdir(data_folder) if file.endswith('.csv')]

# Define a pattern to extract parts of the filenames
filename_pattern = re.compile(r'^(?P<pattern_name>[a-z]+)_?(?P<infill_percentage>\d+)%_(?P<action_type>\w+)\.csv$')

# List to hold extracted data
csv_data = []

# Extract data from filenames
for filename in csv_files_list:
    match = filename_pattern.match(filename)
    if match:
        # Extract pattern name, infill percentage, and action type from each filename
        csv_data.append({
            'filename': filename,
            'pattern_name': match.group('pattern_name'),
            'infill_percentage': int(match.group('infill_percentage')),
            'action_type': match.group('action_type')
        })

# Create a DataFrame
df_csv_data = pd.DataFrame(csv_data)

# Function to load data from CSV
def load_csv_data(filename):
    # Load the data
    return pd.read_csv(f'values/{filename}')

# Plotting function
def plot_csv_data(pattern_names, infill_percentages, action_types):
    plt.figure(figsize=(10, 6))  # Set the figure size

    for index, row in df_csv_data.iterrows():
        if row['pattern_name'] in pattern_names and row['infill_percentage'] in infill_percentages and row['action_type'] in action_types:
            temp_df = load_csv_data(row['filename'])
            # Assuming the single column of values is named 'values'
            # You might need to adjust this if the column has a different name
            plt.plot(temp_df.index, temp_df[temp_df.columns[0]], label=f"{row['pattern_name']} {row['infill_percentage']}% {row['action_type']}")

    plt.title('Plot of CSV Data')
    plt.xlabel('Layer')
    plt.ylabel('Area mm2')
    plt.grid(True)

    # Adjust legend to be outside to the right
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

    plt.show()

# Widget setup
pattern_widget = widgets.SelectMultiple(options=list(df_csv_data['pattern_name'].unique()), description="Patterns")
infill_widget = widgets.SelectMultiple(options=list(df_csv_data['infill_percentage'].unique()), description="Infill %")
action_widget = widgets.SelectMultiple(options=list(df_csv_data['action_type'].unique()), description="Action Type")

ui = widgets.HBox([pattern_widget, infill_widget, action_widget])
out = widgets.interactive_output(plot_csv_data, {'pattern_names': pattern_widget, 'infill_percentages': infill_widget, 'action_types': action_widget})

display(ui, out)


## [🏠 Home](../../../welcomePage.ipynb)